# Part 01
----
In this section, for the most part, we only check the integrity of the datasets. Although there is no reason to believe that the datasets are not clean or not ready to use, we do perform the following tasks just to be extra cautious.

## Read the files

In [146]:
# kaggle instacart competition

In [147]:
path = 'data/'

In [148]:
df_aisles = pd.read_csv(path + 'aisles.csv')
df_departments = pd.read_csv(path + 'departments.csv')
df_orders_prior = pd.read_csv(path + 'order_products__prior.csv')
df_orders_train = pd.read_csv(path + 'order_products__train.csv')
df_orders = pd.read_csv(path + 'orders.csv')
df_products = pd.read_csv(path + 'products.csv')

## How do datasets look like?

In [149]:
df_aisles.head(10)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [150]:
df_departments.head(10)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [151]:
df_orders_prior.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [152]:
df_orders_train.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


In [153]:
df_orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [154]:
df_products.head(10)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


## Are there any null values?

In [155]:
df_aisles.isnull().any()

aisle_id    False
aisle       False
dtype: bool

In [156]:
df_departments.isnull().any()

department_id    False
department       False
dtype: bool

In [157]:
df_orders_prior.isnull().any()

order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool

In [158]:
df_orders.isnull().any()

order_id                  False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
dtype: bool

In [159]:
df_products.isnull().any()

product_id       False
product_name     False
aisle_id         False
department_id    False
dtype: bool

In [160]:
# we see that there is null value in orders dataset
# which may just be the first orders for customers

## What do null values mean?

In [161]:
print('number of orders without days since prior order: {}'.format(
    df_orders.loc[(df_orders.days_since_prior_order.isnull())].shape[0]
))
print('number of unique users: {}'.format(df_orders.user_id.nunique()))

number of orders without days since prior order: 206209
number of unique users: 206209


In [162]:
df_orders.loc[(df_orders.days_since_prior_order.isnull()) & (df_orders.order_number != 1)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order


In [163]:
# this confirms that all orders without days_since_prior_order null are first orders

## Are all train and test orders last orders?

In [164]:
%%time
df_orders_max = df_orders.groupby('user_id').max()

Wall time: 1min 35s


In [165]:
# as it should be, the number of orders marked train or test is equal to the
# number of users in the dataset
df_orders.loc[(df_orders.eval_set == 'train') | (df_orders.eval_set == 'test')] \
    .set_index('user_id').loc[:, ['order_number']].size

206209

In [166]:
# see if the series of last orders and the series of train and test orders are exactly the same
df_orders_max.order_number.equals(df_orders.loc[(df_orders.eval_set == 'train') | (df_orders.eval_set == 'test')]
    .set_index('user_id').order_number)

True

In [167]:
# this shows that all "train" and "test" orders are the last orders of each customer

In [168]:
print('number of train orders: {}'.format(df_orders.loc[df_orders.eval_set == 'train'].shape[0]))
print('number of test orders: {}'.format(df_orders.loc[df_orders.eval_set == 'test'].shape[0]))
print('number of unique users: {}'.format(df_orders.user_id.nunique()))

number of train orders: 131209
number of test orders: 75000
number of unique users: 206209


In [169]:
# all users are divided into one of two groups